In [2]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re
import os


def extract_arabic_text_from_epub(epub_path):
    book = epub.read_epub(epub_path)
    arabic_text = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_body_content()
            soup = BeautifulSoup(content, 'html.parser')

            for header in soup.find_all(['header', 'footer', 'nav']):
                header.decompose()

            for element in soup.find_all(attrs={"class": re.compile(r"header|footer", re.I)}):
                element.decompose()
            for element in soup.find_all(attrs={"id": re.compile(r"header|footer", re.I)}):
                element.decompose()

            text = soup.get_text()

            arabic_pattern = re.compile(r'[\u0600-\u06FF]+')
            arabic_matches = arabic_pattern.findall(text)
            arabic_text.extend(arabic_matches)

    return ' '.join(arabic_text)


PATH = os.getcwd()

with open('./nagib_ds.txt' , 'w+', encoding='utf-8') as fp:
    for file in os.listdir(os.path.join(PATH,'ePubs')):
        try:
            epub_path =os.path.join(PATH,'ePubs' , file)
            arabic_text = extract_arabic_text_from_epub(epub_path)
            c = 0
            for i in arabic_text[600:]:
                if i == ' ':
                    break
                else:
                    c+=1
            arabic_text = arabic_text[600+c:] + '\n' #skip first pages , and cut on whole word(space)
            fp.write(arabic_text)
            
            print(file)
        except:
            continue
